https://keras.io/api/applications/

In [ ]:
!pip install -i https://test.pypi.org/simple/ xin-util

In [ ]:
from model_trainingtime_prediction.env_detect import gpu_features
gpu_features().get_features()

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization
import numpy as np
import time
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
np.random.seed(1000)

In [ ]:
import psutil
print(f'current memory usage {psutil.virtual_memory().percent}%')

In [ ]:
# for valid
def valid_padding_output(input_size, kernel_size, stride):
  pos = kernel_size
  output = 1
  while True:
    pos += stride
    output += 1
    if pos+stride > input_size:
      break
  padding = -(input_size - pos)
  return output

# for same
def same_padding_output(input_size, kernel_size, stride):
  if stride == 1:
    return input_size
  else:
    pos = 1
    output = 1
    while True:
      pos += stride
      output += 1
      if pos+stride > input_size:
        break
    padding = pos+kernel_size-1 - input_size
    return output

def get_data_shape_flow_keras(input_shape, conv_model_obj, start_from=1, up_to=3, conv_weight=1, pool_weight=1):
  multiplications = []
  shape_flow = []
  dense_shapes = []
  input_shape = conv_weight*np.array(input_shape[start_from:up_to])
  shape_flow.append(input_shape)
  conv_shape_flow = []
  polling_shape_flow = []
  conv_shape_flow.append(input_shape)
  for idx, layer_data in enumerate(conv_model_obj.get_config()['layers']):
    layer_name = layer_data['class_name']
    layer_config = layer_data['config']
    if layer_name=='Conv2D' or layer_name=='SeparableConv2D':
      filters = layer_config['filters']
      kernel_size = layer_config['kernel_size'][0]
      strides = layer_config['strides'][0]
      padding_method = layer_config['padding']
      previous_channels = input_shape[-1]
      if padding_method == 'same':
        output = same_padding_output(input_shape[0], kernel_size, strides)
        input_shape = [output, output, filters]
        conv_shape_flow.append(conv_weight*np.array(input_shape[start_from:up_to]))
        shape_flow.append(conv_weight*np.array(input_shape[start_from:up_to]))
        muls = kernel_size * kernel_size * previous_channels * output * output
        multiplications.append(muls)
      else:
        output = valid_padding_output(input_shape[0], kernel_size, strides)
        input_shape = [output, output, filters]
        conv_shape_flow.append(conv_weight*np.array(input_shape[start_from:up_to]))
        shape_flow.append(conv_weight*np.array(input_shape[start_from:up_to]))
        muls = kernel_size * kernel_size * previous_channels * output * output
        multiplications.append(muls)
    if layer_name=='MaxPooling2D' or layer_name=='AveragePooling2D':
      kernel_size = layer_config['pool_size'][0]
      strides = layer_config['strides'][0]
      padding_method = layer_config['padding']
      if padding_method == 'same':
        output = same_padding_output(input_shape[0], kernel_size, strides)
        input_shape = [output, output, input_shape[-1]]
        polling_shape_flow.append(pool_weight*np.array(input_shape[start_from:up_to]))
        shape_flow.append(pool_weight*np.array(input_shape[start_from:up_to]))
      else:
        output = valid_padding_output(input_shape[0], kernel_size, strides)
        input_shape = [output, output, input_shape[-1]]
        polling_shape_flow.append(pool_weight*np.array(input_shape[start_from:up_to]))
        shape_flow.append(pool_weight*np.array(input_shape[start_from:up_to]))
    if layer_name=='ZeroPadding2D':
        w_padding_size = layer_config['padding'][0]
        h_padding_size = layer_config['padding'][1]
        input_shape = [input_shape[0]+np.sum(w_padding_size), input_shape[1]+np.sum(h_padding_size), input_shape[-1]]
        polling_shape_flow.append(pool_weight*np.array(input_shape[start_from:up_to]))
        shape_flow.append(pool_weight*np.array(input_shape[start_from:up_to]))
    if layer_name=='Cropping2D':
        w_cropping_size = layer_config['cropping'][0]
        h_cropping_size = layer_config['cropping'][1]
        input_shape = [input_shape[0]-np.sum(w_cropping_size), input_shape[1]-np.sum(h_cropping_size), input_shape[-1]]
        polling_shape_flow.append(pool_weight*np.array(input_shape[start_from:up_to]))
        shape_flow.append(pool_weight*np.array(input_shape[start_from:up_to]))
    
    if layer_name=='Dense':
      dense_shapes.append(layer_config['units'])
  return shape_flow, conv_shape_flow, polling_shape_flow, dense_shapes, multiplications


In [ ]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph
def get_flops(model, batch_size=None):
    if batch_size is None:
        batch_size = 1

    real_model = tf.function(model).get_concrete_function(tf.TensorSpec([batch_size] + model.inputs[0].shape[1:], model.inputs[0].dtype))
    frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(real_model)

    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph,
                                            run_meta=run_meta, cmd='op', options=opts)
    return flops.total_float_ops

In [ ]:
optimizers = ["sgd", "rmsprop", "adam", "adadelta", "adagrad", "adamax", "nadam", "ftrl"]
losses = ["mae", "mape", "mse", "msle", "poisson", "categorical_crossentropy"]

In [ ]:
input_shape = (75, 75, 3)
classes = 10
batch_size = 32
epochs = 5
truncate_from = 2
trials = 2
optimizer = 'rmsprop'
loss = 'categorical_crossentropy'

In [ ]:
LeNet_5 = Sequential()

LeNet_5.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
LeNet_5.add(AveragePooling2D())

LeNet_5.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
LeNet_5.add(AveragePooling2D())

LeNet_5.add(Flatten())

LeNet_5.add(Dense(units=120, activation='relu'))

LeNet_5.add(Dense(units=84, activation='relu'))

LeNet_5.add(Dense(units=classes, activation = 'softmax'))


In [ ]:
#Instantiation
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=96, input_shape=input_shape, kernel_size=(11,11), strides=(4,4), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))

#5th Convolutional Layer
AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
AlexNet.add(Flatten())
# 1st Fully Connected Layer
AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
# Add Dropout to prevent overfitting
AlexNet.add(Dropout(0.4))

#2nd Fully Connected Layer
AlexNet.add(Dense(4096))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#3rd Fully Connected Layer
AlexNet.add(Dense(1000))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('relu'))
#Add Dropout
AlexNet.add(Dropout(0.4))

#Output Layer
AlexNet.add(Dense(classes))
AlexNet.add(BatchNormalization())
AlexNet.add(Activation('softmax'))

#Model Summary
#AlexNet.summary()


In [ ]:
Xception = tf.keras.applications.Xception(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)



In [ ]:
VGG16 = tf.keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)



In [ ]:
VGG19 = tf.keras.applications.VGG19(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)



In [ ]:
ResNet50 = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax"
)


In [ ]:
ResNet101 = tf.keras.applications.ResNet101(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax"
)


In [ ]:
ResNet152 = tf.keras.applications.ResNet152(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax"
)


In [ ]:
ResNet50V2 = tf.keras.applications.ResNet50V2(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax"
)

In [ ]:
ResNet101V2 = tf.keras.applications.ResNet101V2(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax"
)


In [ ]:
ResNet152V2 = tf.keras.applications.ResNet152V2(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax"
)

In [ ]:
InceptionV3 = tf.keras.applications.InceptionV3(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)

In [ ]:
InceptionResNetV2 = tf.keras.applications.InceptionResNetV2(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)

In [ ]:
MobileNet = tf.keras.applications.MobileNet(
    input_shape=input_shape,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=classes,
    classifier_activation="softmax"
)

In [ ]:
MobileNetV2 = tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)


In [ ]:
DenseNet121 = tf.keras.applications.DenseNet121(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
)

In [ ]:
DenseNet169 = tf.keras.applications.DenseNet169(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
)

In [ ]:
DenseNet201 = tf.keras.applications.DenseNet201(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
)

In [ ]:
NASNetLarge = tf.keras.applications.NASNetLarge(
    input_shape=input_shape,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=classes
)


In [ ]:
NASNetMobile = tf.keras.applications.NASNetMobile(
    input_shape=input_shape,
    include_top=True,
    weights=None,
    input_tensor=None,
    pooling=None,
    classes=classes
)


In [ ]:
EfficientNetB0 = tf.keras.applications.EfficientNetB0(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)

In [ ]:
EfficientNetB1 = tf.keras.applications.EfficientNetB1(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)

In [ ]:
EfficientNetB2 = tf.keras.applications.EfficientNetB2(
    include_top=True,
    weights=None, 
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)

In [ ]:
EfficientNetB3 = tf.keras.applications.EfficientNetB3(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)

In [ ]:
EfficientNetB4 = tf.keras.applications.EfficientNetB4(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)


In [ ]:
EfficientNetB5 = tf.keras.applications.EfficientNetB5(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)

In [ ]:
EfficientNetB6 = tf.keras.applications.EfficientNetB6(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)

In [ ]:
EfficientNetB7 = tf.keras.applications.EfficientNetB7(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=classes,
    classifier_activation="softmax",
)

In [ ]:
# model_list = [LeNet_5, AlexNet, Xception, VGG16, VGG19, ResNet50, ResNet101, ResNet152, ResNet50V2, ResNet101V2, 
#               ResNet152V2, InceptionV3, InceptionResNetV2, MobileNet, MobileNetV2, DenseNet121, DenseNet169, DenseNet201, 
#               NASNetLarge, NASNetMobile, EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, 
#               EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7]

# model_list_name = ['LeNet_5', 'AlexNet', 'Xception', 'VGG16', 'VGG19', 'ResNet50', 'ResNet101', 'ResNet152', 'ResNet50V2', 'ResNet101V2', 
#               'ResNet152V2', 'InceptionV3', 'InceptionResNetV2', 'MobileNet', 'MobileNetV2', 'DenseNet121', 'DenseNet169', 'DenseNet201', 
#               'NASNetLarge', 'NASNetMobile', 'EfficientNetB0', 'EfficientNetB1', 'EfficientNetB2', 'EfficientNetB3', 
#               'EfficientNetB4', 'EfficientNetB5', 'EfficientNetB6', 'EfficientNetB7']


model_list = [LeNet_5, AlexNet, Xception, VGG16, VGG19, ResNet50, ResNet101, ResNet152, ResNet50V2, ResNet101V2, 
              ResNet152V2, InceptionV3, InceptionResNetV2, MobileNet, DenseNet121, DenseNet169, DenseNet201, 
              NASNetLarge, NASNetMobile, EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3, 
              EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7]

model_list_name = ['LeNet_5', 'AlexNet', 'Xception', 'VGG16', 'VGG19', 'ResNet50', 'ResNet101', 'ResNet152', 'ResNet50V2', 'ResNet101V2', 
              'ResNet152V2', 'InceptionV3', 'InceptionResNetV2', 'MobileNet', 'DenseNet121', 'DenseNet169', 'DenseNet201', 
              'NASNetLarge', 'NASNetMobile', 'EfficientNetB0', 'EfficientNetB1', 'EfficientNetB2', 'EfficientNetB3', 
              'EfficientNetB4', 'EfficientNetB5', 'EfficientNetB6', 'EfficientNetB7']

# compile 
for m in model_list:
    m.compile(optimizer=optimizer, loss=loss)

In [ ]:
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.train_start_time = time.time()
        self.epoch_times = []
        self.batch_times = []
        self.epoch_times_detail = []
        self.batch_times_detail = []

    def on_train_end(self, logs={}):
        self.train_end_time = time.time()

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        epoch_time_end = time.time()
        self.epoch_times.append(epoch_time_end - self.epoch_time_start)
        self.epoch_times_detail.append((self.epoch_time_start, epoch_time_end))

    def on_train_batch_begin(self, batch, logs={}):
        self.bacth_time_start = time.time()

    def on_train_batch_end(self, batch, logs={}):
        batch_time_end = time.time()
        self.batch_times.append(batch_time_end - self.bacth_time_start)
        self.batch_times_detail.append((self.bacth_time_start, batch_time_end))

    def relative_by_train_start(self):
        self.epoch_times_detail = np.array(self.epoch_times_detail) - self.train_start_time
        self.batch_times_detail = np.array(self.batch_times_detail) - self.train_start_time
        self.train_end_time = np.array(self.train_end_time) - self.train_start_time

def find_last_dense_layer(keras_model):
  configs = keras_model.get_config()['layers']
  configs.reverse()
  for layer_config in configs:
    try:
      if layer_config['class_name'] == 'Dense':
        return layer_config['config']['units']
      elif layer_config['class_name'] == 'Reshape':
        return layer_config['config']['target_shape'][0]
    except KeyError:
      pass

In [ ]:
times_data = []
setup_data = []
flops_data = []
shape_data = []
conv_shape_data = []
multiplications_data = []


In [ ]:
for idx, m in enumerate(tqdm(model_list)):  
  batch_size_data_batch = []
  batch_size_data_epoch = []
  out_shape = find_last_dense_layer(m)
  print(model_list_name[idx], out_shape)
  x = np.ones((batch_size, *input_shape), dtype=np.float32)
  y = np.ones((batch_size, out_shape), dtype=np.float32)
  for _ in range(trials):
    time_callback = TimeHistory()
    m.fit(
        x,
        y,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[time_callback],
        verbose=False
    )
    times_batch = np.array(time_callback.batch_times) * 1000
    times_epoch = np.array(time_callback.epoch_times) * 1000
    batch_size_data_batch.extend(times_batch)
    batch_size_data_epoch.extend(times_epoch)

  batch_times_truncated = batch_size_data_batch[truncate_from:]
  epoch_times_trancuted = batch_size_data_epoch[truncate_from:]
  recovered_time = [
      np.median(batch_times_truncated)
  ] * truncate_from + batch_times_truncated
  flops = get_flops(m, batch_size=1)
  times_data.append(np.median(batch_times_truncated))
  setup_data.append(np.sum(batch_size_data_batch) - sum(recovered_time))
  flops_data.append(flops)
  shape_flow, conv_shape_flow, polling_shape_flow, dense_shapes, multiplications = get_data_shape_flow_keras(input_shape, m)
  shape_sum = np.sum([np.prod(i) for i in shape_flow])
  shape_data.append(shape_sum)
  conv_sum = np.sum([np.prod(i) for i in conv_shape_flow])
  conv_shape_data.append(conv_sum)
  multiplications_data.append(np.sum(multiplications))


In [ ]:

# plt.figure(figsize=(15, 10))
# for idx, name in enumerate(model_list_name):
#     if name.startswith('Eff'):
#         pass
#     else:
#         plt.text(flops_data[idx], times_data[idx], name, size=15)
#         plt.scatter(flops_data[idx], times_data[idx])
# plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
for idx, name in enumerate(model_list_name):
    if name.startswith('Eff'):
        pass
    else:
        plt.text(conv_shape_data[idx], times_data[idx], name, size=15)
        plt.scatter(conv_shape_data[idx], times_data[idx])
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
for idx, name in enumerate(model_list_name):
    if name.startswith('Eff'):
        pass
    else:
        plt.text(shape_data[idx], times_data[idx], name, size=15)
        plt.scatter(shape_data[idx], times_data[idx])
plt.show()
plt.savefig('withouteff.png')

In [ ]:


plt.figure(figsize=(15, 10))
for idx, name in enumerate(model_list_name):
    if name.startswith('Eff'):
        pass
    else:
        plt.text(multiplications_data[idx], times_data[idx], name, size=15)
        plt.scatter(multiplications_data[idx], times_data[idx])
plt.show()

In [ ]:
unique_layers = set()
for layer in DenseNet121.get_config()['layers']:
    unique_layers.add(layer['class_name'])
unique_layers

In [ ]:
name2time = dict(zip(model_list_name, times_data))

In [ ]:
name2flop = dict([('VGG19', 19.6), ('VGG16', 15.3), ('ResNet152', 11.3), ('ResNet101', 7.6), ('ResNet50', 3.8), ('AlexNet', 0.72)])

In [ ]:
for name,flops in name2flop.items():
    time = name2time[name]
    plt.scatter(flops, time)
    plt.text(flops, time, name, size=15)
    
plt.show()

In [ ]:
# unique_layers = set()
# for layer in DenseNet121.get_config()['layers']:
#     if layer['class_name']=='Concatenate':
#         display(layer)

In [ ]:
data = {'model_list_name': model_list_name, 'times_data': times_data, 'setup_data': setup_data, 'flops_data': flops_data}

In [ ]:
import json
with open('data.json', 'w') as f:
    json.dump(data, f)

In [1]:
from xin_util.ZipAndUnzip import zip_file

In [2]:
zip_file('RealCaseTesting', 'RealCaseTesting.zip')

100%|██████████| 3/3 [00:00<00:00,  6.86it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1000/1000 [00:01<00:00, 832.81it/s]
0it [00:00, ?it/s]
100%|██████████| 4000/4000 [00:05<00:00, 702.55it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1000/1000 [00:01<00:00, 838.95it/s]
0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00,  4.07it/s]
